# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
!pip install gmaps

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities2.csv")
city_weather_df


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Muscat,23.6139,58.5922,62.60,77,20,4.61,OM,1612496929
1,Avarua,-21.2078,-159.7750,84.20,70,20,4.61,CK,1612496904
2,Hervey Bay,-25.2986,152.8535,84.99,57,64,5.99,AU,1612497112
3,Thompson,55.7435,-97.8558,-11.20,77,90,8.05,CA,1612497072
4,Hilo,19.7297,-155.0900,71.60,73,75,16.11,US,1612496910
...,...,...,...,...,...,...,...,...,...
586,Tahoua,14.8888,5.2692,64.40,19,0,6.91,NE,1612497269
587,Lumphat,13.4878,106.9816,88.25,37,35,6.55,KH,1612497269
588,Gemeente Wageningen,51.9667,5.6667,43.00,93,90,4.61,NL,1612497269
589,Rūdsar,37.1376,50.2880,48.20,100,40,2.30,IR,1612497269


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lon"]].astype(float)
humid = city_weather_df["Humidity"].astype(float)

max_humid = city_weather_df["Humidity"].max()
max_humid

100

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
clean_weather_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= 85) & (city_weather_df["Max Temp"] >= 70)\
                                       & (city_weather_df["Cloudiness"] <= 30) & (city_weather_df["Humidity"] <50)]
clean_weather_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,Hong Kong,22.2855,114.1577,75.00,49,20,8.05,HK,1612496919
71,Busselton,-33.6500,115.3333,73.99,41,1,1.99,AU,1612496831
107,Ouadda,8.0777,22.4007,70.84,22,0,8.25,CF,1612497140
130,Tezpur,26.6333,92.8000,72.55,41,0,5.97,IN,1612496941
191,Malakal,9.5334,31.6605,72.63,21,0,13.94,SS,1612497162
201,Kibala,9.1104,18.3463,75.20,23,0,5.37,TD,1612497164
232,Kalas,18.1667,74.8000,74.08,31,0,9.86,IN,1612497173
243,San Rafael,-34.6177,-68.3301,72.39,32,0,3.60,AR,1612497176
257,Bouna,9.2667,-3.0000,74.17,22,25,5.19,CI,1612497179
261,Chatrapur,19.3500,84.9833,74.55,46,0,4.29,IN,1612497180


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:

hotel_df = clean_weather_df[["City", "Country", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lon,Hotel Name
37,Hong Kong,HK,22.2855,114.1577,
71,Busselton,AU,-33.6500,115.3333,
107,Ouadda,CF,8.0777,22.4007,
130,Tezpur,IN,26.6333,92.8000,
191,Malakal,SS,9.5334,31.6605,
201,Kibala,TD,9.1104,18.3463,
232,Kalas,IN,18.1667,74.8000,
243,San Rafael,AR,-34.6177,-68.3301,
257,Bouna,CI,9.2667,-3.0000,
261,Chatrapur,IN,19.3500,84.9833,


In [32]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius" : 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    lon = row['Lon']

    # add keyword to params dict
    params['location'] = f"{lat},{lon}"

    # assemble url and make API request
   
    response = requests.get(base_url, params=params).json()
    

    
    try:
        
        
        hotel_df.loc[index, 'Hotel Name'] = response[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
